In [53]:
!pip3 install imbalanced-learn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [54]:
import warnings
warnings.filterwarnings("ignore")

In [55]:
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [56]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [58]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score,recall_score, precision_score
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve, auc

In [59]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [60]:
df = pd.read_csv('telecom_churn_preprocess_data.csv')

In [61]:
df.head()

,Unnamed: 0,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,streamingmovies,paperlessbilling,monthlycharges,totalcharges,churn,contract_One year,contract_Two year,paymentmethod_Credit card (automatic),paymentmethod_Electronic check,paymentmethod_Mailed check
0,0,0,0,1,0,1,0,1,0,0,...,0,1,29.85,29,0,0,0,0,1,0
1,1,1,0,0,0,34,1,0,0,2,...,0,0,56.95,1889,0,1,0,0,0,1
2,2,1,0,0,0,2,1,0,0,2,...,0,1,53.85,108,1,0,0,0,0,1
3,3,1,0,0,0,45,0,1,0,2,...,0,0,42.30,1840,0,1,0,0,0,0
4,4,0,0,0,0,2,1,0,1,0,...,0,1,70.70,151,1,0,0,0,1,0


In [62]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [63]:
x = df.drop("churn", axis = 1)
y = df['churn']

In [64]:
#We are deleting totalcharges due to high correlation with tenure column.
x.drop('totalcharges', axis=1, inplace=True)

In [65]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state =100 ,stratify=y, test_size = 0.2)
print(y_train.value_counts())

0    4131
1    1485
Name: churn, dtype: int64


In [66]:
scale = MinMaxScaler()
col = x_train.columns

x_train[col] = scale.fit_transform(x_train[col])
x_test[col] = scale.transform(x_test[col])

In [ ]:
#Fit basic model on imbalanced dataset
DT = DecisionTreeClassifier(random_state=0)
cross_val_DT = cross_val_score(DT, x_train, y_train , scoring = 'accuracy' ) 
cross_val_DT_f1 = cross_val_score(DT, x_train, y_train , scoring = 'f1') 

RF = RandomForestClassifier(random_state=0)
cross_val_RF = cross_val_score(RF, x_train, y_train , scoring = 'accuracy' ) 
cross_val_RF_f1 = cross_val_score(RF, x_train, y_train , scoring = 'f1') 

ET = ExtraTreesClassifier(random_state=0)
cross_val_ET = cross_val_score(ET, x_train, y_train , scoring = 'accuracy' ) 
cross_val_ET_f1 = cross_val_score(ET, x_train, y_train , scoring = 'f1') 

MLP = MLPClassifier(random_state=0)
cross_val_MLP = cross_val_score(MLP, x_train, y_train , scoring = 'accuracy' ) 
cross_val_MLP_f1 = cross_val_score(MLP, x_train, y_train , scoring = 'f1') 

GB = GradientBoostingClassifier(random_state=0)
cross_val_GB = cross_val_score(GB, x_train, y_train , scoring = 'accuracy' ) 
cross_val_GB_f1 = cross_val_score(GB, x_train, y_train , scoring = 'f1') 

LR = LogisticRegression(random_state=0)
cross_val_LR = cross_val_score(LR, x_train, y_train , scoring = 'accuracy' ) 
cross_val_LR_f1 = cross_val_score(LR, x_train, y_train , scoring = 'f1') 

KNN = KNeighborsClassifier()
cross_val_KNN = cross_val_score(KNN, x_train, y_train , scoring = 'accuracy' ) 
cross_val_KNN_f1 = cross_val_score(KNN, x_train, y_train , scoring = 'f1') 

In [ ]:
compare_models = [('Decision Tree', cross_val_DT.mean(),cross_val_DT_f1.mean()),
                  ('Random Forest', cross_val_RF.mean(),cross_val_RF_f1.mean()),
                  ('Neural Network', cross_val_MLP.mean(),cross_val_MLP_f1.mean()),
                  ('Extra Tree', cross_val_ET.mean(),cross_val_ET_f1.mean()),
                  ('GradientBoosting', cross_val_GB.mean(),cross_val_GB_f1.mean()),
                  ('Logistic Regression', cross_val_LR.mean(),cross_val_LR_f1.mean()),
                  ('KNN', cross_val_KNN.mean(), cross_val_KNN_f1.mean())]

In [ ]:
predict = pd.DataFrame(data = compare_models, 
                       columns=['Model','Cross_validation_mean(accuracy)', 'Cross_validation_mean(f1_score)'])
predict.style.background_gradient(cmap='YlGn')

In [ ]:
#Random OverSample
x_train_Sample = x_train.copy()
y_train_Sample = y_train.copy()

randOverSample = RandomOverSampler(random_state=100,)
x_train_OverSample, y_train_OverSample = randOverSample.fit_resample(x_train_Sample, y_train_Sample.ravel())

In [ ]:
#Fit basic model on OverSample dataset
DT = DecisionTreeClassifier(random_state=0)
cross_val_DT = cross_val_score(DT, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_DT_f1 = cross_val_score(DT, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

RF = RandomForestClassifier(random_state=0)
cross_val_RF = cross_val_score(RF, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_RF_f1 = cross_val_score(RF, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

ET = ExtraTreesClassifier(random_state=0)
cross_val_ET = cross_val_score(ET, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_ET_f1 = cross_val_score(ET, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

MLP = MLPClassifier(random_state=0)
cross_val_MLP = cross_val_score(MLP, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_MLP_f1 = cross_val_score(MLP, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

GB = GradientBoostingClassifier(random_state=0)
cross_val_GB = cross_val_score(GB, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_GB_f1 = cross_val_score(GB, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

LR = LogisticRegression(random_state=0)
cross_val_LR = cross_val_score(LR, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_LR_f1 = cross_val_score(LR, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

KNN = KNeighborsClassifier()
cross_val_KNN = cross_val_score(KNN, x_train_OverSample, y_train_OverSample , scoring = 'accuracy' ) 
cross_val_KNN_f1 = cross_val_score(KNN, x_train_OverSample, y_train_OverSample , scoring = 'f1') 

In [ ]:
compare_models = [('Decision Tree', cross_val_DT.mean(),cross_val_DT_f1.mean()),
                  ('Random Forest', cross_val_RF.mean(),cross_val_RF_f1.mean()),
                  ('Neural Network', cross_val_MLP.mean(),cross_val_MLP_f1.mean()),
                  ('Extra Tree', cross_val_ET.mean(),cross_val_ET_f1.mean()),
                  ('GradientBoosting', cross_val_GB.mean(),cross_val_GB_f1.mean()),
                  ('Logistic Regression', cross_val_LR.mean(),cross_val_LR_f1.mean()),
                  ('KNN', cross_val_KNN.mean(), cross_val_KNN_f1.mean())]

In [ ]:
predict = pd.DataFrame(data = compare_models, 
                       columns=['Model','Cross_validation_mean(accuracy)', 'Cross_validation_mean(f1_score)'])
predict.style.background_gradient(cmap='YlGn')

In [ ]:
#SMOTE
x_train_sample = x_train.copy()
y_train_sample = y_train.copy()

sm = SMOTE(random_state=100)
x_train_smote, y_train_smote = sm.fit_resample(x_train_sample, y_train_sample.ravel())

In [ ]:
#Fit basic model on OverSample dataset
DT = DecisionTreeClassifier(random_state=0)
cross_val_DT = cross_val_score(DT, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_DT_f1 = cross_val_score(DT, x_train_smote, y_train_smote , scoring = 'f1') 

RF = RandomForestClassifier(random_state=0)
cross_val_RF = cross_val_score(RF, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_RF_f1 = cross_val_score(RF, x_train_smote, y_train_smote , scoring = 'f1') 

ET = ExtraTreesClassifier(random_state=0)
cross_val_ET = cross_val_score(ET, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_ET_f1 = cross_val_score(ET, x_train_smote, y_train_smote , scoring = 'f1') 

MLP = MLPClassifier(random_state=0)
cross_val_MLP = cross_val_score(MLP, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_MLP_f1 = cross_val_score(MLP, x_train_smote, y_train_smote , scoring = 'f1') 

GB = GradientBoostingClassifier(random_state=0)
cross_val_GB = cross_val_score(GB, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_GB_f1 = cross_val_score(GB, x_train_smote, y_train_smote , scoring = 'f1') 

LR = LogisticRegression(random_state=0)
cross_val_LR = cross_val_score(LR, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_LR_f1 = cross_val_score(LR, x_train_smote, y_train_smote , scoring = 'f1') 

KNN = KNeighborsClassifier()
cross_val_KNN = cross_val_score(KNN, x_train_smote, y_train_smote , scoring = 'accuracy' ) 
cross_val_KNN_f1 = cross_val_score(KNN, x_train_smote, y_train_smote , scoring = 'f1') 

In [ ]:
compare_models = [('Decision Tree', cross_val_DT.mean(),cross_val_DT_f1.mean()),
                  ('Random Forest', cross_val_RF.mean(),cross_val_RF_f1.mean()),
                  ('Neural Network', cross_val_MLP.mean(),cross_val_MLP_f1.mean()),
                  ('Extra Tree', cross_val_ET.mean(),cross_val_ET_f1.mean()),
                  ('GradientBoosting', cross_val_GB.mean(),cross_val_GB_f1.mean()),
                  ('Logistic Regression', cross_val_LR.mean(),cross_val_LR_f1.mean()),
                  ('KNN', cross_val_KNN.mean(), cross_val_KNN_f1.mean())]

In [ ]:
predict = pd.DataFrame(data = compare_models, 
                       columns=['Model','Cross_validation_mean(accuracy)', 'Cross_validation_mean(f1_score)'])
predict.style.background_gradient(cmap='YlGn')

In [ ]:
#let's try out Random OverSampling technique on a RandomForest model.
RF = RandomForestClassifier(random_state=0)
RF.fit(x_train_OverSample,y_train_OverSample)
y_pred = RF.predict(x_test)
print ("Accuracy score: ",accuracy_score(y_pred,y_test))
print ("------------------------------")
print ("F1_score: ",f1_score(y_pred,y_test))

In [ ]:
#let's try out Random OverSampling technique on a Extra tree classifier.
ET = ExtraTreesClassifier(random_state=0)
ET.fit(x_train_OverSample,y_train_OverSample)
y_pred = ET.predict(x_test)
print ("Accuracy score: ",accuracy_score(y_pred,y_test))
print ("------------------------------")
print ("F1_score: ",f1_score(y_pred,y_test))

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train_OverSample, y_train_OverSample)

In [ ]:
rf_random.best_params_
'''
{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}
''' 

In [ ]:
rf_random.feature_importances_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(x_train_OverSample, y_train_OverSample)
base_accuracy = evaluate(base_model, x_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test, y_test)

In [ ]:
#feature_importances_.argsort()